# NYC Airbnb Data Cleaning

This notebook demonstrates **step-by-step** how the cleaned dataset (`nyc_airbnb_cleaned.csv`) was produced from the raw scrape (`listings.csv`).

**Files:**

- `nyc_airbnb_raw_data.zip` ➜ contains the original `listings.csv`
- `nyc_airbnb_cleaned_data.zip` ➜ target cleaned file

**Steps:**

1. Load the raw data.
2. Profile missing values and basic statistics.
3. Apply cleaning rules:
   - Drop columns with >50% missing values.
   - Remove rows missing critical identifiers or location fields.
   - Convert data types (e.g., `price` to numeric).
   - Trim whitespace and standardize text columns.
4. Save cleaned data locally (and to a ZIP archive).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from pathlib import Path

RAW_ZIP = Path('/mnt/data/nyc_airbnb_raw_data.zip')
RAW_FILE = 'listings.csv'


In [ ]:
with zipfile.ZipFile(RAW_ZIP) as z:
    with z.open(RAW_FILE) as f:
        raw_df = pd.read_csv(f, low_memory=False)

print(f'Raw shape: {raw_df.shape}')
raw_df.head()

In [ ]:
missing_pct = raw_df.isna().mean().sort_values(ascending=False)
missing_pct.head(20)

## Cleaning Rules

- **Drop columns** with more than 50% missing values.
- **Drop rows** missing critical fields:
  - `id`
  - `neighbourhood_group_cleansed`
  - `room_type`
  - `price`
- **Convert** `price` to numeric (USD).
- **Trim** leading/trailing whitespace in string columns.


In [ ]:
# 1) Drop columns with >50% missing values
thresh = 0.5
cols_to_drop = missing_pct[missing_pct > thresh].index
clean_df = raw_df.drop(columns=cols_to_drop)

# 2) Drop rows missing critical values
critical = ['id', 'neighbourhood_group_cleansed', 'room_type', 'price']
clean_df = clean_df.dropna(subset=critical)

# 3) Clean price column
clean_df['price'] = (clean_df['price']
                     .astype(str)
                     .str.replace('[\$,]', '', regex=True)
                     .astype(float))

# 4) Trim strings in object columns
str_cols = clean_df.select_dtypes(include='object').columns
clean_df[str_cols] = clean_df[str_cols].apply(lambda col: col.str.strip())

print('After cleaning:', clean_df.shape)

In [ ]:
print(f'Columns before: {raw_df.shape[1]}')
print(f'Columns after : {clean_df.shape[1]}')

In [ ]:
OUTPUT_CSV = Path('/mnt/data/nyc_airbnb_cleaned_v2.csv')
clean_df.to_csv(OUTPUT_CSV, index=False)

# Save ZIP
with zipfile.ZipFile('/mnt/data/nyc_airbnb_cleaned_v2.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(OUTPUT_CSV, OUTPUT_CSV.name)

print('Saved cleaned file and ZIP to /mnt/data')

### Next Steps

The cleaned dataset can now be used for:

- Exploratory data analysis
- Visualizations
- Modeling

without worrying about messy column formats or missing values.